In [1]:
import openai
import pandas as pd
import numpy as np
import swifter
import csv


c:\Users\Mathieu\anaconda3\envs\ml_p2_Swiss_AI_Center\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df = pd.read_csv('data/all_metiers.csv', usecols=['titre'])
df.head()

,titre
0,de télégraphe
1,abatjouriste
2,abatteur
3,accessoiriste
4,accompagnant sexuel


In [3]:
salaires = ["produits alimentaires","vêtements, lingerie,chaussures,literies", "industrie textile", "industrie du papier", 
            "arts graphiques", "industrie chimique", "bois,liège,meubles", "pierre et terre", "industrie des métaux",  
            "horlogerie","bijouterie,gravure,frappe","construction,charpenterie","gaz,eau,éléctricité","commerce de gros",
            "commerce de détail","banques,établissements financiers","assurances privées","agences,locations,consultations",
            "affaires immobilières,location","bureaux de consultation","hôtellerie,restauration","transports","administration publique",
            "réparations", "vachers célibataires","employés pour tous,célibataires","employées pour le ménage et la ferme",
            "journaliers,entretien","journalières,entretien"]

In [4]:
#Load the api_key
path_key = "../key_openai/key.txt"
try:
    with open(path_key, 'r') as file:
        api_key = file.read().strip()  
except Exception as e:
    print(f"error : {e}")

In [ ]:
client = openai.Client(api_key=api_key, base_url="https://fmapi.swissai.cscs.ch/")

output_file = "classification_results.csv"

with open(output_file, mode='w', newline='', encoding='utf-8') as f:
    writer = csv.writer(f)
    writer.writerow(["job", "answer"])  

def classify_job(job):
    system_message = {
        "role": "system",
        "content": (
            f"You'll be given a job and you'll have to classify it according to the nearest job in a list we give you: {salaires}. Answer only with the profession in which you classify the job you are given"
        ),
    }
    answer = 'not a job'
    idx = 0 

    while answer not in salaires:
        idx += 1
        if idx == 10:
            answer = None
            break

        user_message = {
            "role": "user",
            "content": f"{job}"
        }

        res = client.chat.completions.create(
            model="meta-llama/Llama-3.3-70B-Instruct",
            messages=[system_message, user_message],
            max_tokens=40,
            stream=True,
        )

        answer = ""
        for chunk in res:
            if len(chunk.choices) > 0 and chunk.choices[0].delta.content:
                answer += chunk.choices[0].delta.content

    with open(output_file, mode='a', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow([job, answer])

    return answer

#df['classification'] = df['titre'].swifter.apply(classify_job)


Pandas Apply: 100%|██████████| 11389/11389 [1:19:23<00:00,  2.39it/s]  
